In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import evaluation
import full_model
import data_aug
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

In [2]:
DATA_DIR = './data/working'

labels = pd.read_pickle("{}/labels.pkl".format(DATA_DIR))

with np.load("{}/splits.npz".format(DATA_DIR)) as splits:
    train_idx = splits['train_idx']
    test_idx = splits['test_idx']

X = np.load("{}/X_cqt.npz".format(DATA_DIR))['X']
X_train = X[train_idx, :]
X_test = X[test_idx, :]

Y = np.load("{}/Y.npz".format(DATA_DIR))['Y']
Y_train = Y[train_idx, :]
Y_test = Y[test_idx, :]

aug = np.load("{}/data_aug.npz".format(DATA_DIR))

with np.load("{}/data_aug.npz".format(DATA_DIR)) as aug:
    X_train = np.vstack((X_train, aug['X']))
    Y_train = np.vstack((Y_train, aug['Y']))

X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

Y_train = torch.from_numpy(np.argmax(Y_train, axis=1))
Y_test = torch.from_numpy(np.argmax(Y_test, axis=1))

In [ ]:
import full_model
import model_tester
import importlib
importlib.reload(full_model)
importlib.reload(model_tester)

os.environ["CUDA_VISIBLE_DEVICES"]="2"
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

losses = model_tester.train_model(X_train, Y_train, X_test, Y_test, full_model.ConvBiLstm(), weight_decay=0.0005,
                         device=device, resume='data/output/checkpoint.pth.tar')

loading checkpoint 'data/output/checkpoint.pth.tar'
ConvBiLstm(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(10, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(10, 10, kernel_size=(10, 10), stride=(2, 2), padding=(4, 4))
  (dense): Conv2d(10, 48, kernel_size=(72, 1), stride=(1, 1))
  (lstm): GRU(48, 48, num_layers=2, batch_first=True, bidirectional=True)
  (conv_pool): Conv2d(1, 10, kernel_size=(1, 75), stride=(1, 1))
  (fc1): Linear(in_features=96, out_features=48, bias=True)
  (fc3): Linear(in_features=48, out_features=24, bias=True)
)
epoch 2 loss: 3.0710602404112337
Train accuracy = 0.24831420373027258
Test accuracy = 0.4136298421807747
